In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from sklearn.model_selection import train_test_split

# import dependencies
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils, to_categorical

import matplotlib.pyplot as plt
%matplotlib inline
# Any results you write to the current directory are saved as output.

In [3]:
train_fpath = "../input/train.csv"
test_fpath = "../input/test.csv"
train_df = pd.read_csv(train_fpath)
test_df = pd.read_csv(test_fpath)

In [4]:
train_df.head()

In [5]:
test_df.head()

In [6]:
#print(train_df.shape)
def np_array_from_dataframe(df, istrain):
    n_rows = df.shape[0]
    arr_list = []
    labels = []
    for row_index in range(n_rows):
        pixels = None
        if istrain:
            pixels =df.iloc[row_index, 1:]
            labels.append(df.iloc[row_index, 0])
        else:
            pixels = df.iloc[row_index, :]
        arr_list.append(np.array(pixels).reshape(-1,28,28,1))
    transformed_arr = np.array(arr_list).reshape(-1,28,28,1)
    transformed_labels = to_categorical(np.array(labels).reshape(-1,1), 10)
    return transformed_arr, transformed_labels if istrain else transformed_arr

In [7]:
# get data from csv
data_x, data_y = np_array_from_dataframe(train_df, istrain=True)
print(data_x.shape)
print(data_y.shape)

In [14]:
# split data into train and validation sets
train_x, valdn_x, train_y, valdn_y = train_test_split(data_x, data_y, test_size=0.1)
print("train_x shape", train_x.shape)
print("train_y shape", train_y.shape)

print("valdn_x shape", valdn_x.shape)
print("valdn_y shape", valdn_y.shape)

In [15]:
test_x,_ = np_array_from_dataframe(test_df, istrain=False)
print("test_x shape", test_x.shape)

In [16]:
# CNN model
model = Sequential()

# -----------------------------------------------------------------------------------
# conv 1
model.add(Conv2D(16, (3,3), input_shape=(28,28,1))) # input- 28X28X3, output- 25,25,16
model.add(BatchNormalization(axis=3))
# relu 1
model.add(Activation('relu'))

# conv 2
model.add(Conv2D(16, (3,3)))        # input-25,25,16, output- 22,22,16
model.add(BatchNormalization(axis=3))
# relu 2
model.add(Activation('relu'))     

# max pool 1
model.add(MaxPooling2D(pool_size=(2,2))) # input - 22,22,16, output

# --------------------------------------------------------------------------------------
# conv 3
model.add(Conv2D(32, (3,3))) 
model.add(BatchNormalization(axis=3))
# relu 3
model.add(Activation('relu'))

# conv 4
model.add(Conv2D(32, (3,3)))  
model.add(BatchNormalization(axis=3))
# relu 4
model.add(Activation('relu'))     

# max pool 2
model.add(MaxPooling2D(pool_size=(2,2)))
# ---------------------------------------------------------------------------------------

# flatten
model.add(Flatten())

# fc layer 1
model.add(Dense(512, activation='relu'))

# fc layer 2
model.add(Dense(256, activation='relu'))

# fc layer 3 
model.add(Dense(128, activation='relu'))

# fc layer 4
model.add(Dense(10, activation='softmax'))


In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
model.summary()

In [19]:
model_hist= model.fit(x=train_x,y=train_y, epochs=10, batch_size= 32, validation_data=(valdn_x, valdn_y), verbose=1)

In [39]:
#model.(valdn_x, valdn_y, batch_size=32, verbose=1)
predictions = model.predict(test_x, batch_size=32, verbose=1)
computed_predictions = np.argmax(predictions, axis=1)

In [40]:
# plot variation of cost with epochs
#print(model_hist.history)
plt.plot(model_hist.history['loss'])
plt.plot(model_hist.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('model loss')
plt.legend(['train', 'test'])
plt.show()

In [41]:
# plot variation of accuracy with epochs
plt.plot(model_hist.history['acc'])
plt.plot(model_hist.history['val_acc'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('model accuracy')
plt.legend(['train', 'test'])
plt.show()

In [31]:
# compute max value from predictions

#print(computed_predictions.shape)

In [42]:
submissions_df = pd.DataFrame(data=computed_predictions, columns=['label'])
#submissions_df['ImageId'] = submissions_df['index'] +1
submissions_df.reset_index(inplace=True)
submissions_df['index'] = submissions_df['index'] + 1
submissions_df.rename(columns={'index': 'ImageId'}, inplace=True)
submissions_df.to_csv('submission.csv', index=False)
submissions_df.head()